In [18]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from tqdm import tqdm
tqdm.pandas()

In [2]:
items = pd.read_parquet('../data/processed/items.parquet.gzip')
users = pd.read_parquet('../data/processed/users.parquet.gzip')

In [3]:
users.head(3)

,id,photo,sex,city,country,likes,weighted_age
0,11946,https://sun6-22.userapi.com/s/v1/ig2/KGV4KtNi0...,1,Санкт-Петербург,Россия,8,35
1,62102,https://sun6-21.userapi.com/s/v1/ig2/iu0M-ERxn...,1,Москва,Россия,9,35
2,662418,https://sun6-21.userapi.com/s/v1/if2/gxSDA6_Q6...,2,Hong Kong,Китай,16,35


In [3]:
items.head()

,id,owner_id,date,text,likes,reposts,attachments,likers,unique_id
0,24666,-196395451,2023-04-07 12:05:01,топ wb😻,80,48,['https://sun6-21.userapi.com/impg/ttKRrIk8GGM...,"[233973847, 272906420, 688902085, 722284040, 4...",24666-196395451
1,24658,-196395451,2023-04-06 18:05:00,серьги wb🦋,43,18,['https://sun6-23.userapi.com/impg/RRRCvHmln_a...,"[625436056, 331203972, 367510803, 360551718, 2...",24658-196395451
2,24642,-196395451,2023-04-05 18:05:00,топ wb💗,108,107,['https://sun6-22.userapi.com/impg/O3fHlE6heq0...,"[272906420, 653691291, 275786983, 534111598, 2...",24642-196395451
3,24637,-196395451,2023-04-05 16:05:00,платье wb🤤,63,141,['https://sun6-21.userapi.com/impg/TEbkmBMtxDN...,"[532433339, 164663002, 543132528, 381103880, 2...",24637-196395451
4,24616,-196395451,2023-04-04 18:45:00,гель для бровей wb😍,76,78,['https://sun6-21.userapi.com/impg/2KajpEpvHU5...,"[704829093, 422933353, 482795060, 525614604, 5...",24616-196395451


In [4]:
items_np = items[['unique_id', 'likers']].to_numpy()

In [5]:
def get_interactions(item_np):
    unique_id = item_np[0]
    likers = item_np[1]
    return list(zip([unique_id] * len(likers), likers, [1.0] * len(likers)))

interactions = []
for item_np in tqdm(items_np): 
    interactions.extend(get_interactions(item_np))

100%|██████████| 20191/20191 [00:01<00:00, 16763.46it/s]


In [20]:
interactions = pd.DataFrame(interactions)
interactions = interactions.rename(columns={0:'post_id', 1:'user_id', 2:'rating'})
interactions.rating = interactions.rating.astype(np.bool_)
interactions.user_id = interactions.user_id.astype(np.int32)
interactions = interactions[['user_id', 'post_id', 'rating']]

In [21]:
# check that we can actually use int32 for user_id
int(interactions.user_id.max()).bit_length()

30

In [22]:
interactions.to_parquet('../data/processed/interactions.parquet.gzip', compression='gzip') 